In [13]:
import urllib.request
from bs4 import BeautifulSoup
from datetime import datetime

# Hàm lấy danh sách bài viết từ trang chủ (single page)
def collect_articles_from_page(page_url, key='h3'):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:11.0) Gecko/20100101'}
    request = urllib.request.Request(page_url, headers=headers)
    
    response = urllib.request.urlopen(request)
    page_html = response.read()
    
    soup = BeautifulSoup(page_html, 'html.parser')
    
    # Tìm tất cả các thẻ <h3> chứa link bài báo
    article_tags = soup.find_all('h3')
    
    article_data = []
    seen_urls = set()  # Set to track unique URLs
    
    for tag in article_tags:
        link = tag.find('a', href=True)
        if link:
            href = link['href']
            title = link.get('title')  # Extract title from 'title' attribute
            
            # Kiểm tra số từ trong tiêu đề (ít nhất 6 từ) và loại bỏ URL trùng
            if title and len(title.split()) >= 6:
                if href.startswith('/'):
                    href = 'https://cafebiz.vn' + href  # Build full URL
                
                if href not in seen_urls:  # Check for duplicates
                    article_data.append({'title': title, 'url': href})
                    seen_urls.add(href)  # Add URL to seen set to avoid duplicates
    
    return article_data

# URL của trang bạn muốn crawl
page_url = 'https://cafebiz.vn/vi-mo.chn'

# Lấy tất cả các tiêu đề và URL bài viết từ trang hiện tại với tiêu đề có ít nhất 6 từ
article_data_list = collect_articles_from_page(page_url)

# In thông tin tiêu đề và URL đã thu thập
for article in article_data_list:
    print(f"title: {article['title']}")
    print(f"url: {article['url']}")

Title: Vòng xoay rộng gấp 10 lần mặt sân Mỹ Đình, sẽ có nhà ga Metro, nhà hàng, trung tâm thương mại
URL: https://cafebiz.vn/vong-xoay-rong-gap-10-lan-mat-san-my-dinh-se-co-nha-ga-metro-nha-hang-trung-tam-thuong-mai-176241004082035368.chn
Title: "Nữ hoàng mía đường" kiến nghị phát voucher cho người dân để kích cầu tiêu dùng
URL: https://cafebiz.vn/nu-hoang-mia-duong-kien-nghi-phat-voucher-cho-nguoi-dan-de-kich-cau-tieu-dung-176241005091501683.chn
Title: Hàng trăm nghìn tấn 'báu vật' của Campuchia tiếp tục đổ về Việt Nam - mặt hàng gì mà nước ta chi tỷ USD gom hàng?
URL: https://cafebiz.vn/hang-tram-nghin-tan-bau-vat-cua-campuchia-tiep-tuc-do-ve-viet-nam-mat-hang-gi-ma-nuoc-ta-chi-ty-usd-gom-hang-176241004140711772.chn
Title: Gạo ST25 ào ạt tăng giá, mỗi kg cõng thêm 3.500 đồng
URL: https://cafebiz.vn/gao-st25-ao-at-tang-gia-moi-kg-cong-them-3500-dong-176241004094840814.chn
Title: Thu nhập bình quân (PPP) Việt Nam từng chưa bằng 1/2 Philippines, 1/3 Indonesia, nay vượt xa Philippines, c

In [1]:
import urllib.request
from bs4 import BeautifulSoup
from datetime import datetime

# Hàm lấy chi tiết từ từng bài báo
def extract_article_details(article_url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:11.0) Gecko/20100101'}
        request = urllib.request.Request(article_url, headers=headers)
        
        # Gửi yêu cầu lấy trang bài báo
        response = urllib.request.urlopen(request)
        article_html = response.read()
        
        # Parse bài báo
        soup = BeautifulSoup(article_html, 'html.parser')
        
        # Lấy tiêu đề
        title = soup.find('h1').get_text(strip=True) if soup.find('h1') else None
        
        # Lấy nội dung từ <h2> và <div class="detail-content">
        sapo = soup.find('h2', class_='sapo')
        content_sapo = sapo.get_text(strip=True) if sapo else ""
        
        detail_content = soup.find('div', class_='detail-content')
        content_paragraphs = detail_content.find_all('p') if detail_content else []
        content_body = "\n".join([para.get_text(strip=True) for para in content_paragraphs])
        
        # Kết hợp <h2> và <div class="detail-content">
        content = content_sapo + "\n" + content_body if content_body else content_sapo
        
        # Lấy ngày đăng bài từ <span class="time">
        time_span = soup.find('span', class_='time')
        published_date = time_span.get_text(strip=True) if time_span else None
        
        # Lấy tác giả
        author_info = soup.find('strong', class_='detail-author').get_text(strip=True) if soup.find('strong', class_='detail-author') else "Unknown"
        content = content.rsplit('.', 1)[0].strip()
        

        # Ngày thu thập dữ liệu
        collected_date = datetime.now().strftime('%Y-%m-%d')
        
        return {
            "title": title,
            "url": article_url,
            "content": content,
            "published_date": published_date,
            "author_info": author_info,
            "collected_date": collected_date
        }
    except Exception as e:
        print(f"Could not retrieve article {article_url}: {e}")
        return None

# Hàm lấy danh sách bài viết từ trang theo ngày
def collect_articles_from_page(page_url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:11.0) Gecko/20100101'}
    request = urllib.request.Request(page_url, headers=headers)
    
    response = urllib.request.urlopen(request)
    page_html = response.read()
    
    soup = BeautifulSoup(page_html, 'html.parser')
    
    # Tìm tất cả các thẻ <h3> chứa link bài báo
    article_tags = soup.find_all('h3')
    
    article_data = []
    seen_urls = set()  # Set to track unique URLs
    
    for tag in article_tags:
        link = tag.find('a', href=True)
        if link:
            href = link['href']
            title = link.get('title')  # Extract title from 'title' attribute
            
            # Kiểm tra số từ trong tiêu đề (ít nhất 6 từ) và loại bỏ URL trùng
            if title and len(title.split()) >= 6:
                if href.startswith('/'):
                    href = 'https://cafebiz.vn' + href  # Build full URL
                
                if href not in seen_urls:  # Check for duplicates
                    article_details = extract_article_details(href)  # Crawl article details
                    if article_details:
                        article_data.append(article_details)
                    seen_urls.add(href)  # Add URL to seen set to avoid duplicates
    
    return article_data

# URL theo ngày bạn muốn crawl
specific_date_url = 'https://cafebiz.vn/xem-theo-ngay-c176114-4-10-2024.chn'

# Lấy tất cả các bài viết với đầy đủ chi tiết
article_data_list = collect_articles_from_page(specific_date_url)

# In thông tin chi tiết các bài viết đã thu thập
for article in article_data_list:
    print(article)

{'title': "Hàng trăm nghìn tấn 'báu vật' của Campuchia tiếp tục đổ về Việt Nam - mặt hàng gì mà nước ta chi tỷ USD gom hàng?", 'url': 'https://cafebiz.vn/hang-tram-nghin-tan-bau-vat-cua-campuchia-tiep-tuc-do-ve-viet-nam-mat-hang-gi-ma-nuoc-ta-chi-ty-usd-gom-hang-176241004140711772.chn', 'content': 'Hiện Việt Nam đang thu gom đến 98% sản lượng của Campuchia.\nẢnh minh họa\nViệt Nam sở hữu một mặt hàng xuất khẩu đứng đầu thế giới 16 năm liên tiếp là hạt điều với 80% tổng sản lượng điều nhân trên toàn cầu. Tuy nhiên bên cạnh nguồn cung trong nước, nước ta đang phải phụ thuộc lớn vào nhập khẩu để đáp ứng đủ sản lượng chế biến phục vụ cho xuất khẩu.\nTheo số liệu thống kê của Tổng cục Hải quan, nhập khẩu hạt điều của Việt Nam trong tháng 8 đạt hơn 211.000 tấn với trị giá hơn 283 triệu USD, giảm 27,5% về lượng và giảm 18,2% về trị giá so với tháng trước đó. Lũy kế 8 tháng đầu năm nước ta đã nhập khẩu hơn 1,9 triệu tấn hạt điều, kim ngạch đạt hơn 2,4 tỷ USD, giảm 4,1% về lượng và giảm 2,2% về

In [2]:
import urllib.request
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

# Hàm lấy chi tiết từ từng bài báo
def extract_article_details(article_url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:11.0) Gecko/20100101'}
        request = urllib.request.Request(article_url, headers=headers)
        
        # Gửi yêu cầu lấy trang bài báo
        response = urllib.request.urlopen(request)
        article_html = response.read()
        
        # Parse bài báo
        soup = BeautifulSoup(article_html, 'html.parser')
        
        # Lấy tiêu đề
        title = soup.find('h1').get_text(strip=True) if soup.find('h1') else None
        
        # Lấy nội dung từ <h2> và <div class="detail-content">
        sapo = soup.find('h2', class_='sapo')
        content_sapo = sapo.get_text(strip=True) if sapo else ""
        
        detail_content = soup.find('div', class_='detail-content')
        content_paragraphs = detail_content.find_all('p') if detail_content else []
        content_body = "\n".join([para.get_text(strip=True) for para in content_paragraphs])
        
        # Kết hợp <h2> và <div class="detail-content">
        content = content_sapo + "\n" + content_body if content_body else content_sapo
        
        # Lấy ngày đăng bài từ <span class="time">
        time_span = soup.find('span', class_='time')
        published_date = time_span.get_text(strip=True) if time_span else None
        
        # Lấy tác giả
        author_info = soup.find('strong', class_='detail-author').get_text(strip=True) if soup.find('strong', class_='detail-author') else "Unknown"
        content = content.rsplit('.', 1)[0].strip()
        

        # Ngày thu thập dữ liệu
        collected_date = datetime.now().strftime('%Y-%m-%d')
        
        return {
            "title": title,
            "url": article_url,
            "content": content,
            "published_date": published_date,
            "author_info": author_info,
            "collected_date": collected_date
        }
    except Exception as e:
        print(f"Could not retrieve article {article_url}: {e}")
        return None

# Hàm lấy danh sách bài viết từ trang theo ngày
def collect_articles_from_page(page_url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:11.0) Gecko/20100101'}
    request = urllib.request.Request(page_url, headers=headers)
    
    response = urllib.request.urlopen(request)
    page_html = response.read()
    
    soup = BeautifulSoup(page_html, 'html.parser')
    
    # Tìm tất cả các thẻ <h3> chứa link bài báo
    article_tags = soup.find_all('h3')
    
    article_data = []
    seen_urls = set()  # Set to track unique URLs
    
    for tag in article_tags:
        link = tag.find('a', href=True)
        if link:
            href = link['href']
            title = link.get('title')  # Extract title from 'title' attribute
            
            # Kiểm tra số từ trong tiêu đề (ít nhất 6 từ) và loại bỏ URL trùng
            if title and len(title.split()) >= 6:
                if href.startswith('/'):
                    href = 'https://cafebiz.vn' + href  # Build full URL
                
                if href not in seen_urls:  # Check for duplicates
                    article_details = extract_article_details(href)  # Crawl article details
                    if article_details:
                        article_data.append(article_details)
                    seen_urls.add(href)  # Add URL to seen set to avoid duplicates
    
    return article_data

# Hàm tạo URL theo ngày
def create_url_for_date(date):
    date_str = date.strftime("%d-%m-%Y")  # Định dạng ngày
    url = f'https://cafebiz.vn/xem-theo-ngay-c176114-{date_str}.chn'
    return url

# Hàm lấy tất cả bài viết trong khoảng thời gian
def collect_articles_between_dates(start_date, end_date):
    current_date = start_date
    all_articles = []
    
    while current_date <= end_date:
        url = create_url_for_date(current_date)
        print(f"Collecting articles for date: {current_date.strftime('%d-%m-%Y')}")
        
        # Thu thập bài viết từ trang theo ngày
        articles = collect_articles_from_page(url)
        all_articles.extend(articles)
        
        # Tăng ngày hiện tại lên 1
        current_date += timedelta(days=1)
    
    return all_articles

# Đặt khoảng thời gian mà bạn muốn crawl
start_date = datetime(2024, 10, 1)
end_date = datetime(2024, 10, 5)

# Lấy tất cả các bài viết trong khoảng thời gian
article_data_list = collect_articles_between_dates(start_date, end_date)

# In thông tin chi tiết các bài viết đã thu thập
for article in article_data_list:
    print(article)


{'title': 'Chi tiết vị trí đặt ga và hướng đi của đường sắt tốc độ cao Bắc-Nam 67 tỷ USD', 'url': 'https://cafebiz.vn/chi-tiet-vi-tri-dat-ga-va-huong-di-cua-duong-sat-toc-do-cao-bac-nam-67-ty-usd-176241001185454145.chn', 'content': 'Báo cáo nghiên cứu tiền khả thi đã mô tả hướng tuyến đường sắt tốc độ cao trên trục Bắc -Nam.\n\nTheo Báo cáo nghiên cứu tiền khả thi Dự án đường sắt tốc độ cao trên trục Bắc - Nam, tuyến đường sắt tốc độ cao bố trí 23 ga hành khách theo nguyên tắc:\nPhù hợp với hiện trạng, quy hoạch phát triển của địa phương; đặt tại khu vực trung tâm kinh tế, chính trị các địa phương, tiếp cận khu vực trung tâm đô thị, khu vực quy hoạch có tiềm năng tạo ra không gian phát triển mới, khai thác có hiệu quả nguồn lực quỹ đất (từ Hà Nội đến TPHCM có 19 đô thị có quy mô 500.000 dân trở lên với 2 đô thị đặc biệt và 17 đô thị loại I); bảo đảm khả năng kết nối tốt với hệ thống giao công cộng; bảo đảm khai thác có hiệu quả hạ tầng, phương tiện.\nDự án sẽ có điểm đầu tại ga Ngọc Hồ